In [21]:
import pandas as pd
from steps import data_ingestion as di
from steps import data_transformation as dt
from steps import model_training as mt
import mlflow_exp as mle

In [16]:
import mlflow
from mlflow import MlflowClient


In [7]:
file_path = 'C:/Users/USER/Desktop/python project/MLFLOW/Real Loan Application/data/Applicant-details.csv'

# get file
df = di.data_ingest(file_path)

df.head()

,Applicant_ID,Annual_Income,Applicant_Age,Work_Experience,Marital_Status,House_Ownership,Vehicle_Ownership(car),Occupation,Residence_City,Residence_State,Years_in_Current_Employment,Years_in_Current_Residence,Loan_Default_Risk
0,75722,9657655,76,0,single,rented,no,Psychologist,Jalandhar,Punjab,0,12,0
1,80185,9259353,37,18,single,rented,no,Petroleum_Engineer,Bally,West_Bengal,12,11,0
2,19865,1509721,66,8,single,rented,no,Drafter,Indore,Madhya_Pradesh,4,12,0
3,76700,5867312,43,1,single,owned,no,Chartered_Accountant,Kurnool[18],Andhra_Pradesh,1,13,1
4,92992,7223191,44,9,single,rented,no,Air_traffic_controller,Asansol,West_Bengal,9,13,0


In [8]:
# cleanup the file
X_train, y_train, X_test, y_test = dt.cleaning_train_pipeline(df)



In [10]:
params = {
    'max_iter': 100,
    'class_weight': 'balanced'
}

In [11]:
# train the model

model, score_train, score_valid, signature = mt.train_model(X_train, y_train, X_test, y_test, params)

In [13]:
score_valid

0.4832869080779944

In [15]:
signature

inputs: 
  [Tensor('float64', (-1, 11))]
outputs: 
  [Tensor('int64', (-1,))]

In [18]:
# set up mlflow client

client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

In [19]:
# MLFLOW SETUP

# CREATING EXPERIMENT


# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the application loan prediction project "
    "This experiment contains the models for loan application"
    "Also this experiment is for learning purpose"
)


# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "loan_application_project",
    "team": "syamil",
    "project_quarter": "Q3-2024",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Loan_Application_Models", tags=experiment_tags
)


In [27]:
# Log the result into mlflow


# setting up the param
exp_param = {
    'name' : 'Loan_Application_Models',
    'run_name': 'second_experiment',
    'artifact_path': 'loan_application_model',
    'model_name': 'Logistic_Regressionv2', 
    'signature': signature
}

model_params = params
model_params['ways'] = 'normalize'

metrics = {
    'training_accuracy': score_train,
    'test_accuracy': score_valid
}

mle.mlflow_logging(exp_param, model, params, metrics)



Successfully registered model 'Logistic_Regressionv2'.
2024/03/23 16:58:14 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Logistic_Regressionv2, version 1
Created version '1' of model 'Logistic_Regressionv2'.


In [23]:
model_params

{'max_iter': 100, 'class_weight': 'balanced', 'ways': 'normal'}

In [28]:
# Test Prediction

import mlflow
logged_model = 'runs:/14ce1075b1f64c9d80f0bdbeb081a4d2/loan_application_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [ ]:
# Predict on a Pandas DataFrame.
import pandas as pd

In [47]:
# test sample data

sample = X_test[1].reshape(-1, 11)


#loaded_model.predict(pd.DataFrame(X_test[0]))

In [48]:
sample = X_test[1].reshape(-1, 11)
sample

array([[ 0.53996447, -0.88150377, -1.51697172,  0.3376169 ,  0.27094972,
        -0.66093323,  0.78790507,  1.06783492,  0.88580077, -1.46646116,
        -0.705783  ]])

In [44]:
pd.DataFrame(X_test[0])

,0
0,-0.791394
1,-1.056995
2,-1.182376
3,0.337617
4,0.270950
5,-0.660933
6,0.044474
7,1.566950
8,-0.286796
9,-0.919201


In [46]:
X_test[0]

array([-0.79139369, -1.05699498, -1.18237607,  0.3376169 ,  0.27094972,
       -0.66093323,  0.04447404,  1.56694988, -0.28679615, -0.91920146,
       -0.705783  ])

In [53]:
#####

import mlflow.pyfunc

model_name = 'Logistic_Regression'
model_version = 3

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

model.predict(sample)

array([1], dtype=int64)

In [55]:
# serving mlflow model

from pprint import pprint

# client = MlflowClient()
for mv in client.search_model_versions("name='Logistic_Regression'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1711184250368,
    'current_stage': 'Archived',
    'description': '',
    'last_updated_timestamp': 1711184606655,
    'name': 'Logistic_Regression',
    'run_id': '126b3e0b128e4284a5cae4db800a7acd',
    'run_link': '',
    'source': 'file:///C:/Users/USER/Desktop/python%20project/MLFLOW/Real%20Loan%20Application/experiment/mlruns/610163555861192253/126b3e0b128e4284a5cae4db800a7acd/artifacts/loan_application_model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '3'}
{   'creation_timestamp': 1711183232673,
    'current_stage': 'Production',
    'description': '',
    'last_updated_timestamp': 1711184606655,
    'name': 'Logistic_Regression',
    'run_id': '14ce1075b1f64c9d80f0bdbeb081a4d2',
    'run_link': '',
    'source': 'file:///C:/Users/USER/Desktop/python%20project/MLFLOW/Real%20Loan%20Application/experiment/mlruns/610163555861192253/14ce1075b1f64c9d80f0bdbeb081a4d2/artifacts/loan_application_model',
